# GPU EXECUTION OPTIMIZATION

In [1]:
import os
#This line is to use a single GPU and not block memory on all GPUs
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"

# IMPORTS

In [2]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
from keras import utils
from keras import backend as K
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.losses import categorical_crossentropy

Using TensorFlow backend.


# DATA PREPROCESSING

In [3]:
# Setting paths for training data, testing data and image dimension size 224x224
TRAIN_DIR = '/media/MyDataStor2/jaybaksh/data/ImageNet/alexnet/train'
TEST_DIR = '/media/MyDataStor2/jaybaksh/data/ImageNet/alexnet/test/'
IMG_SIZE = 224

# LABEL EXTRACTION

In [4]:
def create_labels(img):
    label = img.split('.')[-3]
    return label

# TRAINING DATA EXTRACTION

In [5]:
def create_train_data():
    images=[]
    training_seq_labels=[]
    
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = create_labels(img)
        training_seq_labels.append(label)
        image_path = os.path.join(TRAIN_DIR,img)
        # reading the image in grayscale
        img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        # resizing it to 224x224 target size
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        images.append([img])
    
    # We're shuffling in cross-validation so not needed here
    #shuffle(images)
    
    # Converting the sequence labels into integer encoded and then into one-hot encoded
    # Refer https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
    label_Encoder = LabelEncoder()
    training_onehot_labels = to_categorical(label_Encoder.fit_transform(training_seq_labels))
    
    # Saving the training images as a numpy
    training_images = np.empty((len(images), IMG_SIZE, IMG_SIZE))
    for index in tqdm(range(len(images))):
        training_images[index] = images[index][0]
    
    # Data Normalization
    training_images = ((training_images.astype('float32'))/255)
    np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_images.npy', training_images)
    np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_onehot_labels.npy', training_onehot_labels)
    np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_seq_labels.npy', training_seq_labels)
    print("Training images scanned: {}".format(training_images.shape[0]))
    return training_images, training_onehot_labels, training_seq_labels

In [6]:
def extract_test_id(img):
    img_id = int(img.split('.')[0])
    return img_id

In [7]:
def create_test_data():
    images=[]
    test_id=[]
    
    for img in tqdm(os.listdir(TEST_DIR)):
        img_id = extract_test_id(img)
        test_id.append(img_id)
        image_path = os.path.join(TEST_DIR,img)
        # reading the image in grayscale
        img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        # resizing it to 224x224 target size
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        images.append([img])
    
    # We're shuffling in cross-validation so not needed here
    #shuffle(images)
      
    # Saving the test images as a numpy
    test_images = np.empty((len(images), IMG_SIZE, IMG_SIZE))
    for index in tqdm(range(len(images))):
        test_images[index] = images[index][0]
    
    # Data Normalization
    test_images = ((test_images.astype('float32'))/255)
    np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/new_test_images.npy', test_images)
    np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/new_test_id.npy', test_id)
   
    print("Test images scanned: {}".format(test_images.shape[0]))
    return test_images, test_id

In [8]:
#training_images, training_onehot_labels, training_seq_labels = create_train_data()
training_images = np.load('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_images.npy')
training_onehot_labels = np.load('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_onehot_labels.npy')
training_seq_labels = np.load('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_seq_labels.npy')

In [9]:
#test_images, test_id = create_test_data()
test_images = np.load('/media/MyDataStor2/jaybaksh/data/ImageNet/new_test_images.npy')
test_id = np.load('/media/MyDataStor2/jaybaksh/data/ImageNet/new_test_id.npy')

# TRAINING AND VALIDATION DATA SPLIT

In [10]:
x_train, x_test, y_train, y_test = train_test_split(training_images, training_onehot_labels, test_size=0.3, random_state=40)

print("Number of training images and labels: {}, {}".format(x_train.shape, y_train.shape))
print("Number of test images and labels: {}, {}".format(x_test.shape, y_test.shape))

Number of training images and labels: (17500, 224, 224), (17500, 2)
Number of test images and labels: (7500, 224, 224), (7500, 2)


# DATA RESHAPING

In [11]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(([x_train.shape[0], 1, IMG_SIZE, IMG_SIZE]))
    x_test = x_test.reshape(x_test.shape[0], 1, IMG_SIZE, IMG_SIZE)
    input_shape = (1, IMG_SIZE, IMG_SIZE)
    
else:
    x_train = x_train.reshape(([x_train.shape[0], IMG_SIZE, IMG_SIZE, 1]))
    print("Reshaped x_train: {} and type: {}".format(x_train.shape, type(x_train)))
    print("Reshaped y_train: {} and type: {}".format(len(y_train), type(y_train)))
    
    x_test = x_test.reshape(x_test.shape[0], IMG_SIZE, IMG_SIZE, 1)
    print("Reshaped x_test: {} and type: {}".format(x_test.shape, type(x_test)))
    print("Reshaped y_test: {} and type: {}".format(len(y_test), type(y_test)))
    
    input_shape = (IMG_SIZE, IMG_SIZE, 1)

Reshaped x_train: (17500, 224, 224, 1) and type: <class 'numpy.ndarray'>
Reshaped y_train: 17500 and type: <class 'numpy.ndarray'>
Reshaped x_test: (7500, 224, 224, 1) and type: <class 'numpy.ndarray'>
Reshaped y_test: 7500 and type: <class 'numpy.ndarray'>


In [12]:
if K.image_data_format() == 'channels_first':
    test_images = test_images.reshape(([test_images.shape[0], 1, IMG_SIZE, IMG_SIZE]))
    input_shape = (1, IMG_SIZE, IMG_SIZE)
    
else:
    test_images = test_images.reshape(([test_images.shape[0], IMG_SIZE, IMG_SIZE, 1]))
    print("Reshaped test_images: {} and type: {}".format(test_images.shape, type(test_images)))
    input_shape = (IMG_SIZE, IMG_SIZE, 1)

Reshaped test_images: (12500, 224, 224, 1) and type: <class 'numpy.ndarray'>


# MODEL PARAMETERS

In [13]:
num_classes = 2

# MODEL ARCHITECTURE

In [14]:
model = Sequential()

# Conv Block 1
model.add(Conv2D(64, (3, 3), input_shape=(224,224,1), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Conv Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Conv Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Conv Block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Conv Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# FC layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# MODEL COMPILE

In [15]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)       295168    
__________

In [17]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=15,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 17500 samples, validate on 7500 samples
Epoch 1/15
17500/17500 [==============================] - 225s 13ms/step - loss: 0.6934 - acc: 0.5065 - val_loss: 0.6931 - val_acc: 0.5057
Epoch 2/15
17500/17500 [==============================] - 218s 12ms/step - loss: 0.6923 - acc: 0.5154 - val_loss: 0.7034 - val_acc: 0.4971
Epoch 3/15
17500/17500 [==============================] - 218s 12ms/step - loss: 0.6835 - acc: 0.5662 - val_loss: 0.6643 - val_acc: 0.6028
Epoch 4/15
17500/17500 [==============================] - 218s 12ms/step - loss: 0.6578 - acc: 0.6100 - val_loss: 0.6689 - val_acc: 0.6228
Epoch 5/15
17500/17500 [==============================] - 219s 13ms/step - loss: 0.6222 - acc: 0.6541 - val_loss: 0.6231 - val_acc: 0.6560
Epoch 6/15
17500/17500 [==============================] - 220s 13ms/step - loss: 0.5638 - acc: 0.7093 - val_loss: 0.5430 - val_acc: 0.7177
Epoch 7/15
17500/17500 [==============================] - 219s 13ms/step - loss: 0.4842 - acc: 0.7634 - val_loss: 0.4

In [18]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.634514613595
Test accuracy: 0.875066666667


In [19]:
predicted_encoded_labels = model.predict_classes(test_images,batch_size=64,verbose=1)

12500/12500 [==============================] - 41s 3ms/step


In [20]:
predicted_seq_labels=[]
for label in tqdm(predicted_encoded_labels):
    if label == 0:
        predicted_seq_labels.append('cat')
    else:
        predicted_seq_labels.append('dog')

100%|██████████| 12500/12500 [00:00<00:00, 879795.95it/s]


In [21]:
prediction = []
for index in tqdm(range(len(predicted_seq_labels))):
    prediction.append([test_id[index],predicted_seq_labels[index]])

len(prediction)

100%|██████████| 12500/12500 [00:00<00:00, 817093.43it/s]


12500

In [22]:
prediction

[[10110, 'dog'],
 [10613, 'cat'],
 [1900, 'cat'],
 [9434, 'cat'],
 [10876, 'dog'],
 [5489, 'dog'],
 [1369, 'dog'],
 [10987, 'cat'],
 [10647, 'cat'],
 [2654, 'dog'],
 [8147, 'cat'],
 [5116, 'dog'],
 [8668, 'dog'],
 [556, 'cat'],
 [10081, 'cat'],
 [253, 'dog'],
 [10245, 'dog'],
 [4971, 'dog'],
 [10082, 'cat'],
 [776, 'dog'],
 [3092, 'dog'],
 [9473, 'dog'],
 [9838, 'cat'],
 [140, 'cat'],
 [2034, 'cat'],
 [2541, 'dog'],
 [3445, 'dog'],
 [6598, 'cat'],
 [11949, 'cat'],
 [8393, 'cat'],
 [3581, 'dog'],
 [7526, 'dog'],
 [7172, 'cat'],
 [4269, 'cat'],
 [1724, 'dog'],
 [10237, 'cat'],
 [11284, 'cat'],
 [3828, 'dog'],
 [3271, 'cat'],
 [10150, 'cat'],
 [12147, 'dog'],
 [5791, 'cat'],
 [4081, 'cat'],
 [7864, 'dog'],
 [6546, 'dog'],
 [2836, 'cat'],
 [8683, 'dog'],
 [8715, 'dog'],
 [11934, 'dog'],
 [7199, 'cat'],
 [3197, 'cat'],
 [12440, 'dog'],
 [2705, 'cat'],
 [72, 'dog'],
 [11346, 'dog'],
 [3626, 'dog'],
 [9712, 'cat'],
 [2211, 'dog'],
 [4965, 'cat'],
 [4904, 'cat'],
 [2891, 'dog'],
 [11948, 'cat'

In [23]:
model.save('/media/MyDataStor2/jaybaksh/data/ImageNet/VGG16-ImageNet-Softmax.h5')

In [24]:
model.save_weights('/media/MyDataStor2/jaybaksh/data/ImageNet/vgg16-model-weights.h5')

In [25]:
np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/vgg16-prediction.npy',np.array(prediction))